# Strands- Agentic Memory Patterns: Notebook Walkthrough

This notebook accompanies the blog post on Memory Patterns in Agentic AI and demonstrates practical implementations of hierarchical memory, consolidation, and semantic retrieval within an automotive service assistant use case.

### 🔍 What You'll Learn

- How to structure **short-term**, **episodic**, and **long-term** memory for an LLM-powered agent.
- Techniques to consolidate episodic events and promote context to long-term stores.
- How to use **semantic retrieval** from vector and graph stores to reduce hallucinations and enhance contextual reasoning.
- Examples using memory orchestration tools compatible with **Strands**.

### 🚗 Use Case: Automotive Service Center

To make these patterns concrete, we use an AI agent that assists in diagnosing and resolving car service complaints. The agent leverages different memory tiers to recall past interactions, reason over historical data, and ground its responses in semantic context (e.g., similar issues in other vehicles).

In [ ]:
!pip install boto3 --upgrade

In [ ]:
!pip install langgraph langchain langchain_aws

In [ ]:
!pip install --force-reinstall --no-deps --ignore-installed pytz==2023.3

In [ ]:
!pip install strands-agents strands-agents-tools mem0ai opensearch-py faiss-cpu pytz

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install chromadb
!pip install sentence-transformers

In [1]:
%load_ext autoreload
%autoreload 2

## 🧪 Generating Test Data for Memory Stores

In this section, we simulate realistic memory data for the agent to use across different memory tiers. These datasets serve as the foundation for testing memory orchestration, retrieval patterns, and consolidation logic.

### 🗂️ Memory Store Overview

- **Episodic Memory**  
  Stores past service interactions keyed by session, customer, and vehicle identifiers. These are structured as timestamped events such as repair logs, customer complaints, and agent resolutions.

- **Long-Term Memory**  
  Contains aggregated knowledge distilled from episodic memory. Examples include issue summaries per vehicle or service recommendations based on historical frequency.

- **Semantic Store**  
  Two parallel systems are populated for semantic memory:
  - **Vector Store**: Uses embedding-based similarity to match issue descriptions and resolutions.
  - **Graph Store**: Represents vehicle issues, causes, and resolutions as nodes and relationships for traversal-based reasoning.

### 📋 Synthetic Use Case: Automotive Service Logs

We generate structured data such as:
- Multiple repair sessions for the same vehicle (VIN)
- Repeating issues (e.g., battery drain, brake wear)
- Variations in resolutions and cost estimates
- Semantic embedding-ready entries (e.g., "engine knocking on cold start")
- Graph edges representing causal and similarity links between events

These examples are used to:
- Validate hierarchical memory reads
- Test consolidation (e.g., summarizing similar events)
- Run semantic retrieval on issue queries

In [2]:
from agentic_memory.base import BaseCheckPointer,BaseEpisodicStore, BaseLongTermStore
from agentic_memory.implementation import CheckPointerInMemory, EpisodicStoreFile, LongTermStoreFile
from agentic_memory.orchestrator import MultiTierMemoryOrchestrator
import os, json
from agentic_memory.automotive import RepairCostEstimate, AutomotiveKnowledgeToolkit

In [3]:
from gen_data import generate_long_term_data, generate_episodic_data

generate_episodic_data()
generate_long_term_data()

In [4]:
# Initialize memory stores
short_term = CheckPointerInMemory()
episodic = EpisodicStoreFile()
long_term = LongTermStoreFile()

In [5]:
for result in long_term.search("A/C"):
    print(result)

In [ ]:
from agentic_memory.retrievers import SemanticStoreRetrieval, GraphRetrieval
ssr = SemanticStoreRetrieval(long_term)
ssr.build()

In [7]:
from agentic_memory.retrievers import GraphRetrieval
gr = GraphRetrieval(long_term)
gr.build()

# Initialize memory stores

In [8]:
from agentic_memory.base import BaseCheckPointer,BaseEpisodicStore, BaseLongTermStore
from agentic_memory.implementation import CheckPointerInMemory, EpisodicStoreFile, LongTermStoreFile
from agentic_memory.orchestrator import MultiTierMemoryOrchestrator
import os, json
from agentic_memory.automotive import RepairCostEstimate, AutomotiveKnowledgeToolkit

In [9]:
from agentic_memory.retrievers import SemanticStoreRetrieval, GraphRetrieval
ssr = SemanticStoreRetrieval(long_term)
gr = GraphRetrieval(long_term)

Chroma collection loaded or created at semantic_vector_store


In [10]:
# Initialize memory stores
short_term = CheckPointerInMemory()
episodic = EpisodicStoreFile()
long_term = LongTermStoreFile()

In [11]:
#Create Auto toolkit
auto_tool_kit = AutomotiveKnowledgeToolkit()

# Create orchestrator
orchestrator = MultiTierMemoryOrchestrator(short_term, episodic, long_term,ssr)

## Validate Checkpointer memory
In this section we create a session, create entries to checkpoiner memory and save them by ending the session

In [12]:
# Start new session
session_id = orchestrator.create_session()
vin = "5NPE24AF6FH123456"
cust_id = "cust_016"
key = (cust_id, vin)

# Add diagnostic data
short_term.put(session_id, {"code": "C1234", "description": "ABS pump failure"})
short_term.put(session_id, {"code": "B4567", "description": "Low battery voltage"})

orchestrator.end_session(session_id,key)

In [13]:
episodic.get(key), key

([{'v': 1,
   'value': {'service_type': 'Steering Vibration Diagnosis',
    'mileage': 62000,
    'dealer': 'Precision Tire & Alignment',
    'technician_checks': ['Balanced all four wheels and checked for bent rims',
     'Inspected tie rods and control arms for wear',
     'Checked tire tread and inflation',
     'Test drove at highway speeds'],
    'issues_observed': ['Two wheels out of balance, one rim slightly bent',
     'Minor play in right outer tie rod end',
     'Front tires worn unevenly'],
    'customer_agreement': 'Customer approved wheel balancing and tie rod replacement. Deferred rim repair for future visit.',
    'service_notes': 'Balanced wheels, replaced tie rod, rotated tires, steering vibration reduced. Advised customer on rim repair options.',
    'service_date': '2025-07-24T20:17:17.158976+00:00'}},
  {'v': 1, 'value': {'code': 'C1234', 'description': 'ABS pump failure'}},
  {'v': 1, 'value': {'code': 'B4567', 'description': 'Low battery voltage'}}],
 ('cust_016',

## Patterns Implementation in Strands

This section demonstrates how to implement agent memory patterns using the [Strands](https://strandsagents.com) framework. 
Supports:

- **Hierarchical Memory Retrieval**
- **Semantic Context Lookup**
- **Resolution Logging and Cost Estimation**

### Core Features Implemented

- **Session Management**  
  Each agent interaction is scoped to a session keyed by `(customer_id, VIN)`. The session lifecycle is managed via `create_session()` and `end_session()` methods.

- **Hierarchical Memory Access**  
  The tool `get_hierarchical_memory_context` aggregates data from short-term (checkpointer), episodic, and long-term memory layers for rich context recall.

- **Semantic Memory Retrieval**  
  The tool `get_semantic_memory_context` queries a semantic store (vector or graph-based) to retrieve similar past issues from vehicles of the same make/model.

- **Resolution Storage**  
  Final agent responses and cost estimates are stored in short-term memory using `store_resolution`, enabling future recall and consolidation.

- **Tool-Driven Reasoning**  
  The agent follows a multi-step reasoning path using tools like:
  - `get_vehicle_info`: Resolve VIN to make/model/year
  - `estimate_repair_costs`: Generate detailed cost breakdowns
  - `store_resolution`: Persist structured outcomes

In [14]:
from strands import Agent, tool
from typing import Tuple, Any, Dict
import uuid
import json
import random
from typing_extensions import TypedDict, List
from datetime import datetime, timezone

class AutomotiveTool:
    def __init__(self, orchestrator: MultiTierMemoryOrchestrator):
        self.orchestrator = orchestrator
        self.active_sessions = {}  # session_id: (customer_id, vin)

    def create_session(self, customer_id: str, vin: str) -> str:
        """Creates a new session for customer-vehicle interaction"""
        session_id = str(uuid.uuid4())
        self.active_sessions[session_id] = (customer_id, vin)
        return session_id

        
    def end_session(self, session_id: str):
        """Finalizes session and transfers data to episodic memory"""
        if session_id not in self.active_sessions:
            raise ValueError("Invalid session ID")
        
        customer_id, vin = self.active_sessions[session_id]
        key = (customer_id, vin)
        self.orchestrator.end_session(session_id, key)
        del self.active_sessions[session_id]
        return "Session ended"

    @tool
    def get_hierarchical_memory_context(self, session_id: str) -> dict:
        """Retrieves hierarchical memory context for current session. Give the session Id as the input and get information from past incidents and episodes from episodic and long term memory """
        print(f"TOOL[get_hierarchical_memory_context]: session_id: {session_id}")
        if session_id not in self.active_sessions:
            raise ValueError("Invalid session ID")
        
        customer_id, vin = self.active_sessions[session_id]
        key = (customer_id, vin)
        context = self.orchestrator.get_hierarchical_memory(
            session_id=session_id,
            key=key
        )
        return context
    
    @tool
    def get_semantic_memory_context(self,issue_description:str,make:str = None,model:str = None) -> dict:
        """Retrieves semantic memory context for current session. Accepts issue_description:str,make:str = None,model:str = None """
        print(f"TOOL[get_semantic_memory_context]: issue_description: {issue_description}, make: {make}, model : {model} ")
        return orchestrator.search_semantic_store(
            issue_description=issue_description,
            make= make,
            model=model
        )
    
    @tool
    def store_resolution(self, session_id: str, resolution: str, cost_estimate: List[RepairCostEstimate]):
        """Stores final resolution in short-term memory"""
        try:
            if session_id not in self.active_sessions:
                raise ValueError("Invalid session ID")
            
            self.orchestrator.checkpointer.put(session_id, {
                "type": "resolution",
                "text": resolution,
                "cost": cost_estimate,
                "timestamp": datetime.now(timezone.utc).isoformat()
            })
            return "Resolution stored"
        except Exception as e:
            print(e)            

    @tool
    def estimate_repair_costs(self,repair_items: List[str]) -> List[RepairCostEstimate]:
        """
        Accepts a list of repair item strings and returns a list of dicts with cost breakdowns.
        Each dict contains: 'item', 'labor_cost', 'parts_cost', and 'total_cost'.
        """
        print(f"TOOL[estimate_repair_costs]: repair_items: {repair_items}")
        try:
            estimates =  auto_tool_kit.estimate_repair_costs(repair_items)
            return estimates

        except Exception as e:
            print(e)

    @tool
    def get_vehicle_info(self,vin):
        """
        Returns (make, model, year) for a given VIN.
        """
        print(f"TOOL[get_vehicle_info] vin:{vin}")
        return auto_tool_kit.get_vehicle_info(vin)
      
# Create memory tool
memory_tool = AutomotiveTool(orchestrator)

# Define Strands agent
automotive_agent = Agent(
    model="us.amazon.nova-pro-v1:0",
    system_prompt = (
        """
        You are an automotive service assistant. You are provided with session ID, customer ID, VIN, and customer complaints.
        Always:
        1. Search hierarchical memory to get context from past interactions and episodes from long term stores. 
        2. Identify vehicle's model, make and year. 
        3. Search semantic information to get information to resolve issues.  
        4. When you have a list of repair or fix items in your resolution, use the estimate_repair_costs tool to get detailed cost estimates for each item. The tool accepts a list of repair item strings and returns a breakdown of labor and parts costs for each.
        5. Format your final response as a JSON object with the following fields:
           {
             "VIN": "<vehicle identification number>",
             "Issue": "<summary of the customer's complaint or diagnosed issue>",
             "Resolution": "<detailed resolution steps or recommendations>",
             "CostEstimates": [
               {
                 "item": "<repair item>",
                 "labor_cost": <labor cost as float>,
                 "parts_cost": <parts cost as float>,
                 "total_cost": <total cost as float>
               },
               ...
             ]
           }
        4. Use store_resolution for final solutions.
        5. Include cost estimates in USD where possible.
        """        
    ),
    
    tools=[
        memory_tool.get_hierarchical_memory_context,
        memory_tool.store_resolution,
        memory_tool.estimate_repair_costs,
        memory_tool.get_semantic_memory_context,
        memory_tool.get_vehicle_info 
    ]
)

# Example usage
def handle_complaint_strands(customer_id: str, vin: str, complaint: str):
    """Process automotive complaint through Strands agent"""
    # Start new session
    session_id = memory_tool.create_session(customer_id, vin)

    # Generate resolution (using Nova Pro's reasoning)
    response = automotive_agent(
        f"Session Id: {session_id} \n"
        f"Customer complaint: {complaint}\n"
        f"VIN: {vin}\n"
        # "Context from memory:\n"
        # f"{json.dumps(context, indent=2)}\n\n"
         "Generate resolution with cost estimate:"
    )
    
    # End session
    memory_tool.end_session(session_id)
    
    return response

def parse_response(response: str) -> Dict[str, Any]:
    """
    Parses the LLM's JSON response and extracts VIN, Issue, Resolution, and CostEstimates.
    Returns a dictionary with these fields.
    """
    try:
        data = json.loads(response)
        vin = data.get("VIN", "")
        issue = data.get("Issue", "")
        resolution = data.get("Resolution", "")
        cost_estimates = data.get("CostEstimates", [])
        return {
            "VIN": vin,
            "Issue": issue,
            "Resolution": resolution,
            "CostEstimates": cost_estimates
        }
    except (json.JSONDecodeError, TypeError):
        # Fallback: return empty or default structure if parsing fails
        return {
            "VIN": "",
            "Issue": "",
            "Resolution": response,
            "CostEstimates": []
        }

In [15]:
result = handle_complaint_strands(
    customer_id="cust_012",
    vin="2T1BURHE6JC074321",
    complaint="Engine misfire at highway speeds"
)


<thinking> To resolve the customer's complaint about engine misfire at highway speeds, I need to follow these steps:
1. Retrieve hierarchical memory context to check for any past incidents or episodes related to this VIN or customer.
2. Identify the vehicle's make, model, and year using the VIN.
3. Search semantic memory for information related to engine misfires, specifically for the identified make, model, and year.
4. Based on the semantic information, determine a list of potential repair items.
5. Use the estimate_repair_costs tool to get detailed cost estimates for each repair item.
6. Format the final response as a JSON object with the required fields.
7. Store the resolution using the store_resolution tool. </thinking>

Tool #1: get_hierarchical_memory_context

Tool #2: get_vehicle_info
TOOL[get_hierarchical_memory_context]: session_id: f5afa9eb-9abe-4be3-b09f-f8c8e51df61e
TOOL[get_vehicle_info] vin:2T1BURHE6JC074321
<thinking> Based on the retrieved hierarchical memory context,

In [16]:
print(f"Result: {result} ")

Result: {
  "VIN": "2T1BURHE6JC074321",
  "Issue": "Engine misfire at highway speeds",
  "Resolution": "Perform diagnostic tests to identify the specific cause of the engine misfire. Potential repair items include spark plug replacement, ignition coil replacement, fuel injector cleaning, and O2 sensor replacement.",
  "CostEstimates": [
    {
      "item": "Spark plug replacement",
      "labor_cost": 59.45,
      "parts_cost": 203.09,
      "total_cost": 262.54
    },
    {
      "item": "Ignition coil replacement",
      "labor_cost": 74.03,
      "parts_cost": 256.2,
      "total_cost": 330.23
    },
    {
      "item": "Fuel injector cleaning",
      "labor_cost": 184.42,
      "parts_cost": 251.99,
      "total_cost": 436.40999999999997
    },
    {
      "item": "O2 sensor replacement",
      "labor_cost": 192.11,
      "parts_cost": 118.78,
      "total_cost": 310.89
    }
  ]
}
 


## Consolidator Pattern

In [17]:
from agentic_memory.compaction import Consolidator
consolidator = Consolidator(episodic,long_term)
consolidator.consolidate(('cust_789','1HGBH41JXMN109186'))


{
  "service_events": [
    {
      "issue_summary": "Cylinder 2 misfire detected, low compression, fouled spark plug",
      "resolution": "Replaced spark plugs, further testing if issue persists",
      "service_engineer": "Unnamed Technician",
      "service_date": "2025-07-24T20:17:17.153230+00:00",
      "additional_notes": "Customer shown diagnostic results and agreed to repairs.",
      "cost": {
        "parts": 0.0,
        "labor": 0.0,
        "tax": 0.0,
        "total": 0.0
      }
    },
    {
      "issue_summary": "Heavily fouled spark plug and intermittently failing ignition coil in cylinder 2",
      "resolution": "Replaced all spark plugs and faulty ignition coil, cleared codes, test drive performed",
      "service_engineer": "Unnamed Technician",
      "service_date": "2025-07-24T20:17:17.153524+00:00",
      "additional_notes": "Customer approved replacement of all spark plugs and faulty ignition coil.",
      "cost": {
        "parts": 0.0,
        "labor": 0.0,

'Consolidated 10 episodic events for 1HGBH41JXMN109186.'